In [1]:
import datetime
import calendar
import win32com.client as win32
import openpyxl
import pandas as pd
import ctypes
import win32com
import time
import shutil
import warnings
import os
from pathlib import Path
from openpyxl.styles import Font, NamedStyle, Alignment
from dateutil.parser import parse as du_parse
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from datetime import date
import numpy as np
import xlwings as xw
from openpyxl import Workbook
from openpyxl.styles import Font


dd = datetime.date.today().strftime("%d.%m.%Y")
export1 = str(Path.cwd() / f'export1/ForExecutive_{dd}.xlsx') #№1 таблица (отчет Руководителя)
name_s = os.path.basename(export1)
export2 = str(Path.cwd() / f'hand/На ручнике.xlsx') #№2 таблица (Ручник)
name_s = os.path.basename(export2)
export22 = str(Path.cwd() / f'hand/Убрать из потенциала.xlsx') #№2.2 таблица (Удаление потенциала)
name_s = os.path.basename(export22)
export23 = str(Path.cwd() / f'absence/уволенные.xlsx') #№2.3 таблица (Уволенные)
name_s = os.path.basename(export23)
absence1 = str(Path.cwd() / f'absence/основа_bex1.xlsx') #№3.1 таблица (отсутствие 1 неделя)
name_s = os.path.basename(absence1)
absence2 = str(Path.cwd() / f'absence/основа_bex2.xlsx') #№3.2 таблица (отсутствие 2 неделя)
name_s = os.path.basename(absence2)
absence3 = str(Path.cwd() / f'absence/основа_bex3.xlsx') #№3.3 таблица (отсутствие 3 неделя)
name_s = os.path.basename(absence3)
absence4 = str(Path.cwd() / f'absence/основа_bex4.xlsx') #№3.4 таблица (отсутствие 4 неделя)
name_s = os.path.basename(absence4)
shabl = str(Path.cwd() / f'shabl/ВВБ_AntiCOV_v2.xlsx') #№5 таблица (шаблон для формирования результата)
file_cov = str(Path.cwd() /f"file_cov/ВВБ_AntiCOV_v2_{dd}.xlsx")  #сформированный файл

excel = win32.Dispatch("Excel.Application") #интеграция
ex = True  # Поставить False для итогового варианта
excel.Visible = ex
excel.DisplayAlerts = ex
excel.ScreenUpdating = ex

print('загрузка данных из Отчета руководителя')

#чтение таблицы 1 - отчет Руководителя
df1 = pd.read_excel(export1, sheet_name='1',
                   usecols=["Табельный номер", "Дата вакцинации 1 этап (план)", "Дата вакцинации 1 этап (факт)",  "Дата вакцинации 2 этап (план)", "Дата вакцинации 2 этап (факт)", "Однокомпонентная вакцина",
                            "Дата самоподачи данных о медотводе", "Дата отказа от вакцинации",
                            "Срок действия медотвода", "Дата вакцинации 2 этап (план)","Дата выздоровления"])

df1.fillna("-", inplace=True) 
df1['Табельный номер'] = df1['Табельный номер'].astype(int)
df1 = df1.rename(columns={"Табельный номер": "ТН"})

print('DF1 export1 - Done')
lst_t1 = df1.ТН.tolist()
print("список ТН - Done") 
df1 = df1.set_index('ТН') 

#чтение таблицы 2.1 - отчет Ручника
df2 = pd.read_excel(export2, sheet_name='1',
                   usecols=["Табельный номер", "Дата вакцинации 1 этап (факт)", "Дата вакцинации 2 этап (факт)", "Срок действия медотвода"])

df2.fillna("-", inplace=True) 
df2['Табельный номер'] = df2['Табельный номер'].astype(int)
df2 = df2.rename(columns={"Табельный номер": "ТН"})

print('DF2 export2 - Done')
lst_t2 = df2.ТН.tolist()
print("список ТН - Done") 
df2 = df2.set_index('ТН') 

#чтение таблицы 2.2 - отчет исключение
df22 = pd.read_excel(export22, sheet_name='1',
                   usecols=["Табельный номер"])

df22.fillna("-", inplace=True) 
df22['Табельный номер'] = df22['Табельный номер'].astype(int)
df22 = df22.rename(columns={"Табельный номер": "ТН"})

print('DF22 export22 - Done')
lst_t22 = df22.ТН.tolist()
print("список ТН - Done") 
df22 = df22.set_index('ТН') 


#чтение таблицы 2.3 - отчет исключение уволенные
df23 = pd.read_excel(export23, sheet_name='Лист1',
                   usecols=["Сотрудник"])

df23.fillna("-", inplace=True) 
df23['Сотрудник'] = df23['Сотрудник'].astype(int)
df23 = df23.rename(columns={"Сотрудник": "ТН"})

print('DF23 export23 - Done')
lst_t23 = df23.ТН.tolist()
print("список ТН - Done") 
df23 = df23.set_index('ТН') 


#чтение таблицы 3.1 - отсутствие сотрудников
dfa1 = pd.read_excel(absence1, sheet_name='основа',
                   usecols=["Сотрудник", "Б/лист"])

dfa1.fillna("", inplace=True) 
dfa1["Сотрудник"] = dfa1["Сотрудник"].astype(int)
dfa1 = dfa1.rename(columns={"Сотрудник": "ТН"})
dfa1["Отсутствие на сегодня"] = dfa1["Б/лист"]

lst_a1 = dfa1.ТН.tolist()
print("список ТН - Done") 
dfa1 = dfa1.set_index('ТН')
dfa1.fillna("-", inplace=True)
print('DFA1 absence1 - Done')

#чтение таблицы 3.2 - отсутствие сотрудников
dfa2 = pd.read_excel(absence2, sheet_name='основа',
                   usecols=["Сотрудник", "Б/лист"])

dfa2.fillna("", inplace=True) 
dfa2["Сотрудник"] = dfa2["Сотрудник"].astype(int)
dfa2 = dfa2.rename(columns={"Сотрудник": "ТН"})
dfa2["Отсутствие на сегодня"] = dfa2["Б/лист"]

lst_a2 = dfa2.ТН.tolist()
print("список ТН - Done") 
dfa2 = dfa2.set_index('ТН')
dfa2.fillna("-", inplace=True)
print('DFA2 absence2 - Done')

#чтение таблицы 3.3 - отсутствие сотрудников
dfa3 = pd.read_excel(absence3, sheet_name='основа',
                   usecols=["Сотрудник", "Б/лист"])

dfa3.fillna("", inplace=True) 
dfa3["Сотрудник"] = dfa3["Сотрудник"].astype(int)
dfa3 = dfa3.rename(columns={"Сотрудник": "ТН"})
dfa3["Отсутствие на сегодня"] = dfa3["Б/лист"]

lst_a3 = dfa3.ТН.tolist()
print("список ТН - Done") 
dfa3 = dfa3.set_index('ТН')
dfa3.fillna("-", inplace=True)
print('DFA3 absence3 - Done')

#чтение таблицы 3.4 - отсутствие сотрудников
dfa4 = pd.read_excel(absence4, sheet_name='основа',
                   usecols=["Сотрудник", "Б/лист", "Декрет"])

dfa4.fillna("", inplace=True) 
dfa4["Сотрудник"] = dfa4["Сотрудник"].astype(int)
dfa4 = dfa4.rename(columns={"Сотрудник": "ТН"})
dfa4["Отсутствие на сегодня"] = dfa4["Б/лист"]

lst_a4 = dfa4.ТН.tolist()
print("список ТН - Done") 
dfa4 = dfa4.set_index('ТН')
dfa4.fillna("-", inplace=True)
print('DFA4 absence4 - Done')

print('download shablon')
start_time_2 = time.time()  # время запуска
wb = openpyxl.load_workbook(shabl)

ws = wb.active
n = 0
sheets = wb.sheetnames
wsa = wb[sheets[n]]
# print(wsa)
rows = wsa.max_row 
cols = wsa.max_column 
# wsa.sheet_properties.tabColor = '66FF00'
print('Запускается перебор строк')

#выгрузка данных в шаблон wsa из отчета АРМ-навигатора Статус вакцинации

for i in range(2, rows+1):              
    if wsa.cell(row=i, column=2).value in lst_a1:
        wsa.cell(row=i, column=23).value = dfa1.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]
    elif wsa.cell(row=i, column=2).value not in lst_a1:
        continue
        
    if wsa.cell(row=i, column=2).value in lst_a2:
        wsa.cell(row=i, column=23).value = dfa2.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]+dfa1.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]
    elif wsa.cell(row=i, column=2).value not in lst_a2:
        continue
        
    if wsa.cell(row=i, column=2).value in lst_a3:
        wsa.cell(row=i, column=23).value = dfa3.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]+dfa2.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]+dfa1.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]
    elif wsa.cell(row=i, column=2).value not in lst_a3:
        continue
        
    if wsa.cell(row=i, column=2).value in lst_a4:
        wsa.cell(row=i, column=23).value = dfa4.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]+"1"+dfa4.loc[wsa.cell(row=i, column=2).value, "Декрет"]+dfa3.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]+dfa2.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]+dfa1.loc[wsa.cell(row=i, column=2).value, "Отсутствие на сегодня"]
    elif wsa.cell(row=i, column=2).value not in lst_a4:
        continue
        
   
    #if wsa.cell(row=i, column=2).value in lst_a4:
        #if dfa4.loc[wsa.cell(row=i, column=2).value, "Декрет"]!='':
            #wsa.cell(row=i, column=24).value =1
        
    if wsa.cell(row=i, column=23).value !='1' and wsa.cell(row=i, column=23).value !='':
        wsa.cell(row=i, column=24).value =1
    else: wsa.cell(row=i, column=24).value ='-'

for i in range(2, rows+1):          
    if wsa.cell(row=i, column=2).value not in lst_a4:
        if "УРМ" not in wsa.cell(row=i, column=13).value:
            wsa.cell(row=i, column=24).value =1

for i in range(2, rows+1):

    if wsa.cell(row=i, column=2).value in lst_t1:

        wsa.cell(row=i, column=29).value = df1.loc[wsa.cell(row=i, column=2).value, "Однокомпонентная вакцина"]
       
        wsa.cell(row=i, column=25).value = df1.loc[wsa.cell(row=i, column=2).value, "Дата вакцинации 1 этап (факт)"]
        if wsa.cell(row=i, column=25).value != '-' and wsa.cell(row=i, column=25).value != '':
            wsa.cell(row=i, column=25).value = datetime.datetime.strptime(wsa.cell(row=i, column=25).value, "%d.%m.%Y")

        wsa.cell(row=i, column=26).value = df1.loc[wsa.cell(row=i, column=2).value, "Дата вакцинации 1 этап (план)"]
        if wsa.cell(row=i, column=26).value != '-' and wsa.cell(row=i, column=26).value != '':
            wsa.cell(row=i, column=26).value = datetime.datetime.strptime(wsa.cell(row=i, column=26).value, "%d.%m.%Y")

        wsa.cell(row=i, column=27).value = df1.loc[wsa.cell(row=i, column=2).value, "Дата вакцинации 2 этап (факт)"]
        if wsa.cell(row=i, column=27).value != '-' and wsa.cell(row=i, column=27).value != '':
            wsa.cell(row=i, column=27).value = datetime.datetime.strptime(wsa.cell(row=i, column=27).value, "%d.%m.%Y")

        wsa.cell(row=i, column=28).value = df1.loc[wsa.cell(row=i, column=2).value, "Дата вакцинации 2 этап (план)"]
        if wsa.cell(row=i, column=28).value != '-' and wsa.cell(row=i, column=28).value != '':
            wsa.cell(row=i, column=28).value = datetime.datetime.strptime(wsa.cell(row=i, column=28).value, "%d.%m.%Y")

        wsa.cell(row=i, column=30).value = df1.loc[wsa.cell(row=i, column=2).value, "Дата самоподачи данных о медотводе"]
        if wsa.cell(row=i, column=30).value != '-' and wsa.cell(row=i, column=30).value != '':
            wsa.cell(row=i, column=30).value = datetime.datetime.strptime(wsa.cell(row=i, column=30).value, "%d.%m.%Y")

        wsa.cell(row=i, column=31).value = df1.loc[wsa.cell(row=i, column=2).value, "Дата отказа от вакцинации"]
        if wsa.cell(row=i, column=31).value != '-' and wsa.cell(row=i, column=31).value != '':
            wsa.cell(row=i, column=31).value = datetime.datetime.strptime(wsa.cell(row=i, column=31).value, "%d.%m.%Y")
        wsa.cell(row=i, column=32).value = df1.loc[wsa.cell(row=i, column=2).value, "Срок действия медотвода"]
        if wsa.cell(row=i, column=32).value != '-' and wsa.cell(row=i, column=32).value != '':
            wsa.cell(row=i, column=32).value = datetime.datetime.strptime(wsa.cell(row=i, column=32).value, "%d.%m.%Y")
                
        wsa.cell(row=i, column=60).value = df1.loc[wsa.cell(row=i, column=2).value, "Дата выздоровления"]
        if wsa.cell(row=i, column=60).value != '-' and wsa.cell(row=i, column=60).value != '':
            wsa.cell(row=i, column=60).value = datetime.datetime.strptime(wsa.cell(row=i, column=60).value, "%d.%m.%Y")


    if wsa.cell(row=i, column=2).value in lst_t2:
        wsa.cell(row=i, column=25).value = df2.loc[wsa.cell(row=i, column=2).value, "Дата вакцинации 1 этап (факт)"]
        wsa.cell(row=i, column=27).value = df2.loc[wsa.cell(row=i, column=2).value, "Дата вакцинации 2 этап (факт)"]
        wsa.cell(row=i, column=32).value = df2.loc[wsa.cell(row=i, column=2).value, "Срок действия медотвода"]
        if wsa.cell(row=i, column=32).value != '-' and wsa.cell(row=i, column=32).value != '':
            wsa.cell(row=i, column=32).value = datetime.datetime.strptime(wsa.cell(row=i, column=32).value, "%d.%m.%Y")

    
    if wsa.cell(row=i, column=2).value in lst_t1: 
    #заполняем столбец 35-зависит от 27 -ок+  
        #if wsa.cell(row=i, column=27).value != '-':  
            #wsa.cell(row=i, column=35).value=1 
        #elif wsa.cell(row=i, column=25).value != '-' and wsa.cell(row=i, column=27).value == '-' and wsa.cell(row=i, column=29).value=='Да':
            #wsa.cell(row=i, column=35).value=1
        #else: wsa.cell(row=i, column=35).value='-'

        #заполняем столбец 52 и 53 зависит от 25,27,60-ок+

        if wsa.cell(row=i, column=25).value != '-'and wsa.cell(row=i, column=27).value != '-':
             wsa.cell(row=i, column=52).value = max(wsa.cell(row=i, column=25).value, wsa.cell(row=i, column=27).value)
        elif wsa.cell(row=i, column=27).value == '-' and wsa.cell(row=i, column=25).value != '-':
            wsa.cell(row=i, column=52).value = wsa.cell(row=i, column=25).value 
        elif wsa.cell(row=i, column=25).value == '-' and wsa.cell(row=i, column=27).value != '-':
            wsa.cell(row=i, column=52).value = wsa.cell(row=i, column=27).value 
        elif wsa.cell(row=i, column=25).value == '-'and wsa.cell(row=i, column=27).value == '-':
            wsa.cell(row=i, column=52).value = '-'
                
        if wsa.cell(row=i, column=52).value != '-'and wsa.cell(row=i, column=60).value != '-':
            wsa.cell(row=i, column=53).value = max(wsa.cell(row=i, column=52).value, wsa.cell(row=i, column=60).value)
        elif wsa.cell(row=i, column=52).value == '-' and wsa.cell(row=i, column=60).value != '-':
            wsa.cell(row=i, column=53).value = wsa.cell(row=i, column=60).value 
        elif wsa.cell(row=i, column=60).value == '-' and wsa.cell(row=i, column=52).value != '-':
            wsa.cell(row=i, column=53).value = wsa.cell(row=i, column=52).value 
        elif wsa.cell(row=i, column=60).value == '-'and wsa.cell(row=i, column=52).value == '-':
             wsa.cell(row=i, column=53).value = '-'

                  
        #заполняем столбец 48 -зависит от 32 -ок+

        if wsa.cell(row=i, column=32).value != '-':
            arg1 = datetime.datetime.today()
            arg2 = wsa.cell(row=i, column=32).value
            timeD=arg2-arg1+timedelta(1)
            wsa.cell(row=i, column=48).value=timeD.days
        else: wsa.cell(row=i, column=48).value='-'
            
        #заполняем столбец 46 -зависит от 60 -ок+

        if wsa.cell(row=i, column=60).value != '-':
             wsa.cell(row=i, column=46).value = wsa.cell(row=i, column=60).value+timedelta(180)
        else: wsa.cell(row=i, column=46).value = '-'
                
        #заполняем столбец 49 -зависит от 46 -ок+
        
        if wsa.cell(row=i, column=46).value != '-'and wsa.cell(row=i, column=46).value != '':
            wsa.cell(row=i, column=49).value = (wsa.cell(row=i, column=46).value-datetime.datetime.today()+timedelta(1)).days
        else: wsa.cell(row=i, column=49).value ='-'
            
        #заполняем столбец 47 -зависит от 46,32 -ок+

        if wsa.cell(row=i, column=46).value != '-'and wsa.cell(row=i, column=32).value != '-':
            wsa.cell(row=i, column=47).value = max(wsa.cell(row=i, column=46).value, wsa.cell(row=i, column=32).value)
        elif wsa.cell(row=i, column=46).value == '-':
             wsa.cell(row=i, column=47).value = wsa.cell(row=i, column=32).value
        elif wsa.cell(row=i, column=32).value == '-':
            wsa.cell(row=i, column=47).value = wsa.cell(row=i, column=46).value
        else: wsa.cell(row=i, column=47).value =='-'
            
        #заполняем столбец 42 -зависит от 46,47 -ок+

        if wsa.cell(row=i, column=46).value != '-'and wsa.cell(row=i, column=47).value != '-':
            m43 = max(wsa.cell(row=i, column=46).value, wsa.cell(row=i, column=47).value)
            wsa.cell(row=i, column=42).value = (m43-datetime.datetime.today()+timedelta(1)).days
        elif wsa.cell(row=i, column=46).value == '-'and wsa.cell(row=i, column=47).value != '-':
            wsa.cell(row=i, column=42).value = (wsa.cell(row=i, column=47).value-datetime.datetime.today()+timedelta(1)).days 
        elif wsa.cell(row=i, column=47).value == '-'and wsa.cell(row=i, column=46).value != '-':
            wsa.cell(row=i, column=42).value = (wsa.cell(row=i, column=46).value-datetime.datetime.today()+timedelta(1)).days
        elif wsa.cell(row=i, column=47).value == '-'and wsa.cell(row=i, column=46).value == '-':
            wsa.cell(row=i, column=42).value ='-'
            
                            
        #заполняем столбец 54 -зависит от 52 -ок+
        
        if wsa.cell(row=i, column=52).value != '-'and wsa.cell(row=i, column=52).value != '':
            wsa.cell(row=i, column=54).value =184-(datetime.datetime.today()-(wsa.cell(row=i, column=52).value)).days
        else: wsa.cell(row=i, column=54).value='-' 
            
        #заполняем столбец 50 -зависит от 54, 42   -ок +  

        if wsa.cell(row=i, column=42).value != '-' and wsa.cell(row=i, column=54).value != '-':
            m511=wsa.cell(row=i, column=42).value
            m512=184-(datetime.datetime.today()-(wsa.cell(row=i, column=52).value)).days
            m55 = max(m511, m512)
            wsa.cell(row=i, column=50).value =m55
        elif wsa.cell(row=i, column=54).value == '-' and wsa.cell(row=i, column=42).value != '-':
            wsa.cell(row=i, column=50).value = wsa.cell(row=i, column=42).value
        elif wsa.cell(row=i, column=42).value == '-' and wsa.cell(row=i, column=54).value != '-':
            wsa.cell(row=i, column=50).value = wsa.cell(row=i, column=54).value
        elif wsa.cell(row=i, column=54).value == '-'and wsa.cell(row=i, column=42).value == '-':
            wsa.cell(row=i, column=50).value = '-'
   
                   
        #заполняем столбец 56 -зависит от 50  -ок+

        if wsa.cell(row=i, column=50).value!= '-' and wsa.cell(row=i, column=50).value>0 and wsa.cell(row=i, column=50).value<=30:
            wsa.cell(row=i, column=56).value =1
        else: wsa.cell(row=i, column=56).value ='-'
            
        #заполняем столбец 57 -зависит от 50 -ок+

        if wsa.cell(row=i, column=50).value!= '-' and wsa.cell(row=i, column=50).value>0 and wsa.cell(row=i, column=50).value<7:
            wsa.cell(row=i, column=57).value =1
        else: wsa.cell(row=i, column=57).value ='-'
            
        #заполняем столбец 59 -зависит от 50 -ок+

        if wsa.cell(row=i, column=54).value!= '-' and wsa.cell(row=i, column=54).value>0:
            wsa.cell(row=i, column=59).value =1
        else: wsa.cell(row=i, column=59).value ='-'
            
        #заполняем столбец 34 -зависит от 54-ок+ #правильно-184
     
        if wsa.cell(row=i, column=54).value!= '-' and wsa.cell(row=i, column=54).value>(-184):
            wsa.cell(row=i, column=34).value =1   
        #elif wsa.cell(row=i, column=50).value!= '-' and wsa.cell(row=i, column=50).value>=(-82):#!!!!!!!!!!!!!!!!!!!!!!!
            #wsa.cell(row=i, column=35).value =1 
        else: wsa.cell(row=i, column=34).value ='-'
            
        #заполняем столбец 35
        
        if wsa.cell(row=i, column=34).value == 1 and wsa.cell(row=i, column=29).value=='Да':  
            wsa.cell(row=i, column=35).value=1 
        elif wsa.cell(row=i, column=34).value == 1 and wsa.cell(row=i, column=27).value != '-' and wsa.cell(row=i, column=29).value!='Да':
            wsa.cell(row=i, column=35).value=1
        else: wsa.cell(row=i, column=35).value='-'
            
        #заполняем столбец 39 -зависит от 30,47,34, 31, 30,52 -ок+проверить
        
        dataP=datetime.datetime(2050, 12, 31)
        if wsa.cell(row=i, column=47).value!= '-' and wsa.cell(row=i, column=47).value < dataP and wsa.cell(row=i, column=34).value=='-' and wsa.cell(row=i, column=48).value!= '-' and wsa.cell(row=i, column=48).value>0:
            wsa.cell(row=i, column=39).value=1
        elif wsa.cell(row=i, column=47).value!= '-' and wsa.cell(row=i, column=47).value < dataP and wsa.cell(row=i, column=34).value=='-'and wsa.cell(row=i, column=49).value!= '-' and wsa.cell(row=i, column=49).value>(-184):
            wsa.cell(row=i, column=39).value=1
        else: wsa.cell(row=i, column=39).value='-'
                      
                  
        #заполняем столбец 40-зависит от 32,34,39,31,30  -ок+проверить
            
        if wsa.cell(row=i, column=32).value!='-' and wsa.cell(row=i, column=32).value >= dataP and wsa.cell(row=i, column=39).value =='-' and wsa.cell(row=i, column=34).value== '-':
            wsa.cell(row=i, column=40).value=1                   
        else: wsa.cell(row=i, column=40).value='-'
            
        #заполняем столбец 36 -зависит от 39,40,34 -ок+

        if wsa.cell(row=i, column=39).value!= '-'or wsa.cell(row=i, column=40).value != '-':
            wsa.cell(row=i, column=36).value=1
        else:
            wsa.cell(row=i, column=36).value='-' 
        
        #заполняем столбец 41 -зависит от 50,39 -ок+проверить

        if wsa.cell(row=i, column=50).value!= '-' and wsa.cell(row=i, column=50).value<(-14) and wsa.cell(row=i, column=39).value!= '-':
            wsa.cell(row=i, column=41).value =1
        else: wsa.cell(row=i, column=41).value ='-'
            
        #заполняем столбец 45 -зависит от 40,60,30 -ок+!!! изм до -180!!!!!!!!!!
                
        if wsa.cell(row=i, column=40).value==1 and wsa.cell(row=i, column=60).value !='-' and wsa.cell(row=i, column=49).value>(-185):
            wsa.cell(row=i, column=45).value=1
        else: wsa.cell(row=i, column=45).value='-'
            
        #заполняем столбец 37 -зависит от 30,31,52,36 -ок+
            
        if wsa.cell(row=i, column=31).value != '-'and wsa.cell(row=i, column=36).value!= 1 and wsa.cell(row=i, column=34).value != 1:
            wsa.cell(row=i, column=37).value=1
        else: wsa.cell(row=i, column=37).value='-' 
            
         #заполняем столбец 38 -зависит от 36,37,34 -ок+
        
        if wsa.cell(row=i, column=36).value=='-'and wsa.cell(row=i, column=37).value=='-' and wsa.cell(row=i, column=34).value=='-':
            wsa.cell(row=i, column=38).value =1
        else: wsa.cell(row=i, column=38).value ='-'
            
        #заполняем столбец 55 -зависит от 50,36,37,38  -ок+ изменения 15.12!!!!

        if wsa.cell(row=i, column=50).value!= '-' and wsa.cell(row=i, column=50).value<(0) and wsa.cell(row=i, column=34).value==1 and wsa.cell(row=i, column=24).value!=1:
            if wsa.cell(row=i, column=31).value=='-':
                wsa.cell(row=i, column=55).value =1
            if wsa.cell(row=i, column=31).value!='-' and wsa.cell(row=i, column=31).value<wsa.cell(row=i, column=52).value:
                wsa.cell(row=i, column=55).value =1
        else: wsa.cell(row=i, column=55).value ='-'
        
             
        #заполняем столбец 51 и 44 -зависит от 54 -ок?изменения !!!!изм. дней изм до -180!!!!!!!!!!!!
            
        if wsa.cell(row=i, column=54).value!='-' and wsa.cell(row=i, column=54).value>(0) and wsa.cell(row=i, column=34).value==1:
            wsa.cell(row=i, column=51).value=1
        else: wsa.cell(row=i, column=51).value='-'
            
        if wsa.cell(row=i, column=60).value !='-' and wsa.cell(row=i, column=39).value==1 and wsa.cell(row=i, column=49).value>(-185):
            wsa.cell(row=i, column=44).value=1
        else: wsa.cell(row=i, column=44).value='-' 
            
         #заполняем столбец 43 -зависит от 44,47 -ок?

        if wsa.cell(row=i, column=44).value==1 or wsa.cell(row=i, column=45).value==1:
            wsa.cell(row=i, column=43).value=1
        else: wsa.cell(row=i, column=43).value='-'
            
    else:
        wsa.cell(row=i, column=25).value ='-'
        wsa.cell(row=i, column=26).value ='-'
        wsa.cell(row=i, column=27).value ='-'
        wsa.cell(row=i, column=28).value ='-'
        wsa.cell(row=i, column=29).value ='-'
        wsa.cell(row=i, column=30).value ='-'
        wsa.cell(row=i, column=31).value ='-'
        wsa.cell(row=i, column=32).value ='-'
        wsa.cell(row=i, column=60).value ='-' 
        
    if wsa.cell(row=i, column=25).value=='-'and wsa.cell(row=i, column=34).value!=1 and wsa.cell(row=i, column=36).value!=1 and wsa.cell(row=i, column=27).value=='-'and wsa.cell(row=i, column=30).value=='-' and wsa.cell(row=i, column=31).value=='-':
        wsa.cell(row=i, column=38).value =1
        wsa.cell(row=i, column=54).value=(-365)
        #доп. 15.12.2022
    if wsa.cell(row=i, column=38).value==1 and wsa.cell(row=i, column=24).value!=1:
        wsa.cell(row=i, column=55).value=1
    if wsa.cell(row=i, column=38).value==1:
        if wsa.cell(row=i, column=54).value=="-" or wsa.cell(row=i, column=54).value=='':
            wsa.cell(row=i, column=54).value=(-365)            
  
    if wsa.cell(row=i, column=38).value==1 and wsa.cell(row=i, column=55).value==1:
        wsa.cell(row=i, column=58).value=1
    else: continue
        
for i in range(2, rows+1):        
    if wsa.cell(row=i, column=34).value==1 and wsa.cell(row=i, column=51).value==1:
        wsa.cell(row=i, column=33).value="Вакцинирован менее 6 месяцев назад"
    else: continue
for i in range(2, rows+1): 
    if wsa.cell(row=i, column=34).value==1 and wsa.cell(row=i, column=51).value!=1:
        wsa.cell(row=i, column=33).value="Вакцинирован менее 12 месяцев назад"
    else: continue
for i in range(2, rows+1): 
    if wsa.cell(row=i, column=39).value==1 and wsa.cell(row=i, column=43).value!=1:
        wsa.cell(row=i, column=33).value="Временный медоотвод"
    else: continue
for i in range(2, rows+1): 
    if wsa.cell(row=i, column=40).value==1 and wsa.cell(row=i, column=43).value!=1:
        wsa.cell(row=i, column=33).value="Постоянный медоотвод"
    else: continue
for i in range(2, rows+1): 
    if wsa.cell(row=i, column=43).value==1:
        wsa.cell(row=i, column=33).value="Переболел менее 12 месяцев назад"
    else: continue
for i in range(2, rows+1): 
    if wsa.cell(row=i, column=37).value==1:
        wsa.cell(row=i, column=33).value="Отказ"
    else: continue
for i in range(2, rows+1): 
    if wsa.cell(row=i, column=38).value==1:
        wsa.cell(row=i, column=33).value="Не определился"
    else: continue

        
for i in range(2, rows+1):              
    if wsa.cell(row=i, column=2).value in lst_t22:
        wsa.cell(row=i, column=55).value ='-'
        wsa.cell(row=i, column=56).value ='-'
        wsa.cell(row=i, column=57).value ='-'
        wsa.cell(row=i, column=58).value ='-'
for i in range(2, rows+1):              
    if wsa.cell(row=i, column=2).value in lst_t23:
        wsa.cell(row=i, column=55).value ='-'
        wsa.cell(row=i, column=56).value ='-'
        wsa.cell(row=i, column=57).value ='-'
        wsa.cell(row=i, column=58).value ='-'

print('переменные')

wb.save(file_cov)
ext = True
excel.Visible = ext
excel.DisplayAlerts = ext
excel.ScreenUpdating = ext

print(f'save - {file_cov}')

# wb = excel.Workbooks.Open(shabl
print("Создание отчета по просрочке для ЕСР")
dd = datetime.date.today().strftime("%d.%m.%Y")
df=pd.read_excel(f'file_cov/ВВБ_AntiCOV_v2_{dd}.xlsx', sheet_name='Лист1', 
               usecols=["Внутренний e-mail", "Просрочена ревакцинация", "Прогноз ревакцинации на неделю"])

#выгрузка данных в шаблон wsa из Коллективного иммунитета
df["Внутренний e-mail"]=np.where((df["Просрочена ревакцинация"]==1)|(df["Прогноз ревакцинации на неделю"]==1), df["Внутренний e-mail"], np.nan)
df=df[df["Внутренний e-mail"].notnull()]
    
del df["Просрочена ревакцинация"]
del df["Прогноз ревакцинации на неделю"]

df.to_excel(f'ovbECR_ITOG/Обработчик к ревакцинации ЕСР_{dd}.xlsx', index=False)

#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'ovbECR_ITOG/Обработчик к ревакцинации ЕСР_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 43
ws.cell(row=8000, column=1).font= Font(size=12, name ='Calibri')

wb.save(f'ovbECR_ITOG/Обработчик к ревакцинации ЕСР_{dd}.xlsx')

print('Обработчик для ЕСР готов')

dd = datetime.date.today().strftime("%d.%m.%Y") 

df=pd.read_excel(f'file_cov/ВВБ_AntiCOV_v2_{dd}.xlsx', sheet_name='Лист1', 
               usecols=["Просрочена ревакцинация", "Прогноз ревакцинации на неделю", "Табельный номер руководителя", "ФИО"])

df["ФИО"]=np.where((df["Просрочена ревакцинация"]==1)|(df["Прогноз ревакцинации на неделю"]==1), df["ФИО"], np.nan) 
df=df[df["ФИО"].notnull()]
df=df.sort_values(by=["Табельный номер руководителя"])


#выгрузка данных в шаблон wsa из Коллективного иммунитета df["ФИО"]

del df["Просрочена ревакцинация"] 
del df["Прогноз ревакцинации на неделю"]

df["Внутренний e-mail руков"]="" 
df["Внешний e-mail руков"]=""
df["Нумерация"]=""
df["Сцепка1"]=""
df["Сцепка2"]=""
df["Выбор"]=""
df["Сообщение"]=""

df.to_excel(f'ovbRBOSS_ITOG/Обработчик к ревакцинации для руководителей_{dd}.xlsx', index=False)

print("готово1")


spr = str(Path.cwd() / f'spr/Справочник.xlsx') #№1 таблица Справочник
name_s = os.path.basename(spr)
shabl = str(Path.cwd() / f'ovbRBOSS_ITOG/Обработчик к ревакцинации для руководителей_{dd}.xlsx') 
file_cov = str(Path.cwd() /f'ovbRBOSS_ITOG/Обработчик к ревакцинации для руководителей_{dd}.xlsx')  #сформированный файл

excel = win32.Dispatch("Excel.Application") #интеграция
ex = True  # Поставить False для итогового варианта
excel.Visible = ex
excel.DisplayAlerts = ex
excel.ScreenUpdating = ex

print('загрузка данных')

#чтение таблицы 1 
df2 = pd.read_excel(spr, sheet_name='Справочник')

df2.fillna("-", inplace=True) 
df2['Табельный номер руководителя'] = df2['Табельный номер руководителя'].astype(int)
df2 = df2.rename(columns={"Табельный номер руководителя": "ТН"})

print('DF1 chisl - Done')
lst_t1 = df2.ТН.tolist()
print("список ТН - Done") 
df2 = df2.set_index('ТН') 

print('download shablon')
start_time_2 = time.time()  # время запуска
wb = openpyxl.load_workbook(shabl)

ws = wb.active
n = 0
sheets = wb.sheetnames
wsa = wb[sheets[n]]
# print(wsa)
rows = wsa.max_row 
cols = wsa.max_column 
# wsa.sheet_properties.tabColor = '66FF00'
print('Запускается перебор строк')

#выгрузка данных в шаблон wsa 
for i in range(2, rows+1):
    if wsa.cell(row=i, column=2).value in lst_t1:
        wsa.cell(row=i, column=3).value = df2.loc[wsa.cell(row=i, column=2).value, "Внутренний e-mail руков"]
        wsa.cell(row=i, column=4).value = df2.loc[wsa.cell(row=i, column=2).value, "Внешний e-mail руков"]
    else: wsa.cell(row=i, column=3).value=0
        
    if wsa.cell(row=i, column=2).value==wsa.cell(row=i-1, column=2).value:
        wsa.cell(row=i, column=5).value=wsa.cell(row=i-1, column=5).value+1
    else: wsa.cell(row=i, column=5).value=1
    
   # wsa.cell(row=i, column=4).value=str(wsa.cell(row=i, column=4).value)
   # wsa.cell(row=i, column=3).value=str(wsa.cell(row=i, column=3).value)
for i in range(2, rows+1):    
    if wsa.cell(row=i, column=4).value!='' and wsa.cell(row=i, column=5).value!='':
        wsa.cell(row=i, column=5).value=str(wsa.cell(row=i, column=5).value)
        wsa.cell(row=i, column=1).value=str(wsa.cell(row=i, column=1).value)
        wsa.cell(row=i, column=6).value="<strong>"+(wsa.cell(row=i, column=5).value)+"."+" "+(wsa.cell(row=i, column=1).value)+"</strong>"+"<br>"+"<br>"

    if wsa.cell(row=i, column=2).value==wsa.cell(row=i-1, column=2).value and wsa.cell(row=i, column=7).value!="":
        wsa.cell(row=i, column=6).value=str(wsa.cell(row=i, column=6).value)
        wsa.cell(row=i-1, column=7).value=str(wsa.cell(row=i-1, column=7).value)
        wsa.cell(row=i, column=7).value=wsa.cell(row=i-1, column=7).value+wsa.cell(row=i, column=6).value
    else: wsa.cell(row=i, column=7).value=wsa.cell(row=i, column=6).value
        
for i in range(2, rows+1): 
    arg1=str(wsa.cell(row=i+1, column=7).value)
    arg2=str(wsa.cell(row=i, column=7).value)
    if len(arg2)<len(arg1):
        wsa.cell(row=i, column=8).value=0
    else: wsa.cell(row=i, column=8).value=1

for i in range(2, rows+1): 
    if wsa.cell(row=i, column=8).value==1:
        wsa.cell(row=i, column=9).value="Сообщаем о необходимости прохождения ревакцинации и передачи данных в СБЕРДРУГ для сотрудников Вашего подразделения:"+"<br>"+"<br>"+(wsa.cell(row=i, column=7).value)
    else: wsa.cell(row=i, column=9).value=""
print('переменные')


wb.save(file_cov)
ext = True
excel.Visible = ext
excel.DisplayAlerts = ext
excel.ScreenUpdating = ext

print(f'save - {file_cov}')


df=pd.read_excel(f'ovbRBOSS_ITOG/Обработчик к ревакцинации для руководителей_{dd}.xlsx', sheet_name='Sheet1')

df["ФИО"]=np.where((df["Выбор"]!=0), df["ФИО"], np.nan) 
df=df[df["ФИО"].notnull()]

df["ФИО"]=np.where((df["Табельный номер руководителя"]!=0), df["ФИО"], np.nan) 
df=df[df["ФИО"].notnull()]

df["ФИО"]=np.where((df["Внутренний e-mail руков"]!=0), df["ФИО"], np.nan) 
df=df[df["ФИО"].notnull()]

#выгрузка данных в шаблон wsa из Коллективного иммунитета df["ФИО"]

del df["ФИО"] 
del df["Табельный номер руководителя"]
del df["Нумерация"]
del df["Сцепка1"]
del df["Сцепка2"]
del df["Выбор"]

df.to_excel(f'ovbRBOSS_ITOG/Обработчик к ревакцинации для руководителей_{dd}.xlsx', index=False)

#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'ovbRBOSS_ITOG/Обработчик к ревакцинации для руководителей_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 30
ws.column_dimensions['B'].width = 30
ws.column_dimensions['C'].width = 30
ws.cell(row=2000, column=1).font= Font(size=12, name ='Calibri')
ws.cell(row=2000, column=2).font= Font(size=12, name ='Calibri')
ws.cell(row=2000, column=3).font= Font(size=12, name ='Calibri')

wb.save(f'ovbRBOSS_ITOG/Обработчик к ревакцинации для руководителей_{dd}.xlsx')


print("Обработчик к ревакцинации для руководителей готов")

загрузка данных из Отчета руководителя
DF1 export1 - Done
список ТН - Done
DF2 export2 - Done
список ТН - Done
DF22 export22 - Done
список ТН - Done
DF23 export23 - Done
список ТН - Done
список ТН - Done
DFA1 absence1 - Done
список ТН - Done
DFA2 absence2 - Done
список ТН - Done
DFA3 absence3 - Done
список ТН - Done
DFA4 absence4 - Done
download shablon
Запускается перебор строк
переменные
save - C:\Users\19356099\Documents\COVID-19\file_cov\ВВБ_AntiCOV_v2_24.04.2023.xlsx
Создание отчета по просрочке для ЕСР
Обработчик для ЕСР готов
готово1
загрузка данных
DF1 chisl - Done
список ТН - Done
download shablon
Запускается перебор строк
переменные
save - C:\Users\19356099\Documents\COVID-19\ovbRBOSS_ITOG\Обработчик к ревакцинации для руководителей_24.04.2023.xlsx
Обработчик к ревакцинации для руководителей готов
